# 🏥 Notebook 2: Computer Vision in Healthcare

In this notebook, you’ll apply what you learned to a real healthcare problem:  
**detecting pneumonia from chest X-ray images**.  

---

## 🎯 Learning Objectives
By the end of this notebook, you will be able to:
1. Prepare medical images (X-rays) for training a deep learning model.  
2. Apply **transfer learning** using ResNet to a healthcare task.  
3. Evaluate model performance and interpret results.  
4. Reflect on the limitations and ethical considerations of AI in medicine.  

---

## 🏥 Why this matters
- Pneumonia is a leading cause of death worldwide.  
- Radiologists often need to examine thousands of X-rays under time pressure.  
- AI models can help **assist diagnosis** by flagging suspicious cases.  

👉 In this notebook, you’ll **train a CNN to detect pneumonia** using chest X-rays.


## 2.1 Data Preparation

Medical images are often:
- Different sizes (512×512, 1024×1024, etc.)  
- Different intensity ranges (depending on the scanner used)  

We’ll:
1. Resize all images to **224×224**  
2. Convert them to tensors (numerical format)  
3. Normalize pixel values  


In [ ]:
#@title Define Image Transformations
import torchvision.transforms as transforms

image_transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # grayscale X-rays
])

print("✅ Transformations ready!")


In [ ]:
#@title Upload and Preview an X-ray
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt

uploaded = files.upload()

for fname in uploaded.keys():
    img = Image.open(fname).convert("L")  # grayscale
    plt.figure(figsize=(6,3))

    # Original
    plt.subplot(1,2,1)
    plt.imshow(img, cmap="gray")
    plt.title("Original X-ray")

    # Transformed
    transformed = image_transformations(img)
    plt.subplot(1,2,2)
    plt.imshow(transformed[0], cmap="gray")
    plt.title("Transformed X-ray")
    plt.show()


## 2.2 Transfer Learning with ResNet

Instead of building a CNN from scratch, we’ll use **ResNet18**,  
pre-trained on ImageNet, and fine-tune it for pneumonia detection.

Why transfer learning?
- Saves compute time  
- Works well with smaller medical datasets  
- Learns general features (edges, textures) and adapts them to X-rays


In [ ]:
#@title Load Pretrained ResNet Model
import torch.nn as nn
import torchvision.models as models

# Load ResNet18
model = models.resnet18(pretrained=True)

# Modify final layer for binary classification (pneumonia vs normal)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

print("✅ ResNet18 ready for pneumonia classification!")


## 2.3 Model Training

We’ll fine-tune the model on our dataset.  
This may take a while depending on your dataset size.  

👉 For teaching purposes, we’ll show the training loop structure here.


In [ ]:
#@title Define Training Loop (simplified)
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, epochs=2):
    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs} complete ✅")


## 2.4 Model Testing & Evaluation

Now let’s test the model on unseen data and check its performance.  

We’ll calculate:
- Accuracy  
- Confusion matrix  
- Example predictions  

👉 In healthcare, evaluation is **critical** — a wrong prediction can mean a missed diagnosis.


In [ ]:
#@title Evaluate the Model
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())
    acc = accuracy_score(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)
    return acc, cm


## 2.5 Capstone Project

Now it’s your turn 🚀  

- Train your model on the pneumonia dataset  
- Evaluate its performance  
- Visualize some misclassified cases  

💡 **Challenge:**  
What patterns do you notice in the X-rays that were misclassified?  
Could they confuse even a human radiologist?


# 🌍 Ethics & Limitations

AI in healthcare is powerful but not perfect.  
Consider these questions:
- What are the risks of false negatives in pneumonia detection?  
- How might bias in the dataset (e.g., only one hospital’s patients) affect results?  
- Should AI replace radiologists, or assist them?  

---

✅ Congratulations! You’ve built your first medical AI pipeline.  
In the next notebook, we’ll explore **federated learning** to train across hospitals without sharing patient data.
